In [0]:
%scala
//Zadanie 1
import org.apache.spark.sql.types.{IntegerType, StringType, StructType, StructField}

val schema = StructType(Array(
  StructField("imdb_title_id", StringType, true),
  StructField("ordering", IntegerType, true),
  StructField("imdb_name_id", StringType, true),
  StructField("category", StringType, true),
  StructField("job", StringType, true),
  StructField("characters", StringType, true)))

val filePath = "dbfs:/FileStore/tables/Files/actors.csv"

val actorsDf = spark.read.format("csv")
              .option("header", "true")
              .option("inferSchema", "true")
              .load(filePath)

val actorsDfSchema = spark.read.format("csv")
            .option("header", "true")
            .schema(schema)
            .load(filePath)

// display(actorsDf)
// display(actorsDfSchema)
actorsDf.printSchema
actorsDfSchema.printSchema

root
-- imdb_title_id: string (nullable = true)
-- ordering: integer (nullable = true)
-- imdb_name_id: string (nullable = true)
-- category: string (nullable = true)
-- job: string (nullable = true)
-- characters: string (nullable = true)

root
-- imdb_title_id: string (nullable = true)
-- ordering: integer (nullable = true)
-- imdb_name_id: string (nullable = true)
-- category: string (nullable = true)
-- job: string (nullable = true)
-- characters: string (nullable = true)

import org.apache.spark.sql.types.{IntegerType, StringType, StructType, StructField}
schema: org.apache.spark.sql.types.StructType = StructType(StructField(imdb_title_id,StringType,true), StructField(ordering,IntegerType,true), StructField(imdb_name_id,StringType,true), StructField(category,StringType,true), StructField(job,StringType,true), StructField(characters,StringType,true))
filePath: String = dbfs:/FileStore/tables/Files/actors.csv
actorsDf: org.apache.spark.sql.DataFrame = [imdb_title_id: string, ordering: int ... 4 more fields]
actorsDfSchema: org.apache.spark.sql.DataFrame = [imdb_title_id: string, ordering: int ... 4 more fields]

In [0]:
%scala
// Zadanie 2
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

val actorsCols = actorsDf.select("category", "characters")

val schema = StructType(Array(
StructField("category", StringType, true),
StructField("characters", StringType, true)))

val actorsJSON = actorsCols.write.option("header", "true").json("dbfs:/FileStore/tables/Files/file_2.json")

val ff = spark.read.json("dbfs:/FileStore/tables/Files/file_2.json")
display(ff)

val actorsRows = """
[{
"imdb_title_id": "tt0000009",
"ordering": 1,
"imdb_name_id": "nm0063086",
"category": "actress",
"job": "null",
"characters": [
"Miss Geraldine Holbrook (Miss Jerry)"
]
},
{
"imdb_title_id": "tt0000009",
"ordering": 2,
"imdb_name_id": "nm0183823",
"category": "actor",
"job": "null",
"characters": [
"Mr. Hamilton"
]
},
{
"imdb_title_id": "tt0002844",
"ordering": 4,
"imdb_name_id": "nm0137288",
"category": "actress",
"job": "null",
"characters": [
"Lady Beltham",
"maîtresse de Fantômas"
]
}]
"""
 
val schema = StructType(Array(
StructField("imdb_title_id", StringType, true),
StructField("ordering", IntegerType, true),
StructField("imdb_name_id", StringType, true),
StructField("category", StringType, true),
StructField("job", StringType, true),
StructField("characters", StringType, true)))

def convertJSONtoDataFrame(Rows: String, schema: StructType = null): DataFrame = {
val reader = spark.read
Option(schema).foreach(reader.schema) //zapytac, dlaczego Option jest bez . ?
reader.json(sc.parallelize(Array(Rows))) //tworzy RDD - https://sparkbyexamples.com/apache-spark-rdd/how-to-create-an-rdd-using-parallelize/
}

val actorsRowsDf = convertJSONtoDataFrame(actorsRows, schema)
display(actorsRowsDf)

val actorsRowsDfJSON = actorsRowsDf.write.mode("overwrite").json("dbfs:/FileStore/tables/Files/actorsRowsDfJSON.json")


In [0]:
%scala
//Zadanie 4

val brokendata = """
[{
'imdb_title_id': 'tt0110009', 
'ordering': 2, 
'imdb_name_id': 'nm003086', 
'category': 1, 'job': 'null', 
'characters': 'Miss Geraldine Holbrook (Miss Jerry)'
},
{
'imdb_title_id': 123, 
'ordering': 2, 
'imdb_name_id': 'nm003086', 
'category': 1, 
'job': 'null', 
'characters': 'Miss Geraldine Holbrook (Miss Jerry)'
},
{
xxxxxxxxxxxxxxxx
}]
"""
 
val brokendataDf = convertJSONtoDataFrame(brokendata)

val brokendataDfJSON = brokendataDf.write.mode("overwrite").json("dbfs:/FileStore/tables/Files/brokendataDfJSON.json")

val badRecords = spark.read.format("json")
              .option("inferSchema","true")
              .option("badRecordsPath", "dbfs:/FileStore/tables/Files/badrecords")
.load("dbfs:/FileStore/tables/Files/brokendataDfJSON.json")

val permissive = spark.read.format("json")
.option("inferSchema","true")
.option("mode", "PERMISSIVE")
.load("dbfs:/FileStore/tables/Files/brokendataDfJSON.json")
  
val dropmalformed = spark.read.format("json")
.option("inferSchema","true")
.option("mode", "DROPMALFORMED")
.load("dbfs:/FileStore/tables/Files/brokendataDfJSON.json")

val failfast = spark.read.format("json")
.option("inferSchema","true")
.option("mode", "FAILFAST")
.load("dbfs:/FileStore/tables/Files/brokendataDfJSON.json")


In [0]:
%scala
//zadanie 5
val file = actorsDf.write.mode("overwrite").option("header", "true").parquet("dbfs:/FileStore/tables/Files/file1.parquet")

val parquet_file = spark.read.format("parquet")
.option("inferSchema","true")
.load("dbfs:/FileStore/tables/Files/file1.parquet")

val file2 = actorsDf.write.mode("overwrite").option("header", "true").json("dbfs:/FileStore/tables/Files/file2.json")

val json_file = spark.read.format("json")
          .option("inferSchema","true")
          .load("dbfs:/FileStore/tables/Files/file1_2.json")

display(parquet_file)